In [88]:
# !pip install PyPDF2
# !pip install nltk

     |████████████████████████████████| 1.5 MB 6.2 MB/s eta 0:00:01


In [91]:
from PyPDF2 import PdfReader
import re
import nltk
from nltk.corpus import stopwords

In [30]:
reader = PdfReader("sample_pdf/_au_cases_vic_VCAT_2014_1282.pdf")
number_of_pages = len(reader.pages)
page = reader.pages[0]
text = page.extract_text()

In [11]:
from os import listdir
# onlyfiles = [f for f in listdir(mypath) if isfile(join(mypath, f))]
listdir('sample_pdf/')

['_au_cases_vic_VCAT_2019_397.pdf',
 '_au_cases_vic_VCAT_2016_1790.pdf',
 '_au_cases_vic_VCAT_2018_386.pdf',
 '_au_cases_vic_VCAT_2014_1502.pdf',
 '_au_cases_vic_VCAT_2022_29.pdf',
 '_au_cases_vic_VCAT_2017_129.pdf',
 '_au_cases_vic_VCAT_2012_520.pdf',
 '_au_cases_vic_VCAT_2006_2295 (1).pdf',
 '_au_cases_vic_VCAT_2020_1104.pdf',
 '_au_cases_vic_VCAT_2018_1558.pdf',
 '_au_cases_vic_VCAT_2004_1561.pdf',
 '_au_cases_vic_VCAT_2011_1482.pdf',
 '_au_cases_vic_VCAT_2022_1156.pdf',
 '_au_cases_vic_VCAT_2012_284.pdf',
 '_au_cases_vic_VCAT_1998_1 (4).pdf',
 '_au_cases_vic_VCAT_2021_1105.pdf',
 '_au_cases_vic_VCAT_2013_732.pdf',
 '_au_cases_vic_VCAT_2006_1916.pdf',
 '_au_cases_vic_VCAT_2021_116.pdf',
 '_au_cases_vic_VCAT_2011_1086.pdf',
 '_au_cases_vic_VCAT_2022_1208.pdf',
 '_au_cases_vic_VCAT_2016_2065.pdf',
 '_au_cases_vic_VCAT_2017_1544.pdf',
 '_au_cases_vic_VCAT_2013_644.pdf',
 '_au_cases_vic_VCAT_2020_1073.pdf',
 '_au_cases_vic_VCAT_2012_1570.pdf',
 '_au_cases_vic_VCAT_2018_150.pdf',
 '_au_c

In [92]:
stop_words = set(stopwords.words('english'))

In [177]:
case = []
case_numbers = []
order = []
for i,doc in enumerate(listdir('sample_pdf/')):
    print(doc)
    reader = PdfReader(f"sample_pdf/{doc}")
    text = reader.pages[0].extract_text()+reader.pages[1].extract_text()
    case_no = ''
    case_num = i
    is_order = False
    for line in text.split('\n'):
        line = line.strip()
#         print(f"{len(list_line)} {line}")
        list_line = line.split(' ')
        if bool(re.search(r'VCAT REFERENCE NO.', line.upper())):
            case_no = re.search(r'(?<=VCAT REFERENCE NO).*', line.upper()).group(0)
        if line[:5] == 'ORDER' or line[:26] == 'I MAKE THE FOLLOWING ORDER':
            is_order = True
            continue
        if line[:7] =='BETWEEN':
            continue
        if line[:10] == 'APPEARANCE':
            break
        elif is_order and (len(list_line) > 1 or list_line[0].isdigit()):
    #         print(len(list_line))
            if bool(re.search('MEMBER|JUDGE|PRESIDENT', line.upper())) or bool(re.search('PAGE \d OF \d', line.upper())):
    #             print('catch error 1')
                continue
            order.append(' '.join(list_line))
            case.append(case_no)
            case_numbers.append(case_num)
    #     print(line.strip())

_au_cases_vic_VCAT_2019_397.pdf
_au_cases_vic_VCAT_2016_1790.pdf
_au_cases_vic_VCAT_2018_386.pdf
_au_cases_vic_VCAT_2014_1502.pdf
_au_cases_vic_VCAT_2022_29.pdf
_au_cases_vic_VCAT_2017_129.pdf
_au_cases_vic_VCAT_2012_520.pdf
_au_cases_vic_VCAT_2006_2295 (1).pdf
_au_cases_vic_VCAT_2020_1104.pdf
_au_cases_vic_VCAT_2018_1558.pdf
_au_cases_vic_VCAT_2004_1561.pdf
_au_cases_vic_VCAT_2011_1482.pdf
_au_cases_vic_VCAT_2022_1156.pdf
_au_cases_vic_VCAT_2012_284.pdf
_au_cases_vic_VCAT_1998_1 (4).pdf
_au_cases_vic_VCAT_2021_1105.pdf
_au_cases_vic_VCAT_2013_732.pdf
_au_cases_vic_VCAT_2006_1916.pdf
_au_cases_vic_VCAT_2021_116.pdf
_au_cases_vic_VCAT_2011_1086.pdf
_au_cases_vic_VCAT_2022_1208.pdf
_au_cases_vic_VCAT_2016_2065.pdf
_au_cases_vic_VCAT_2017_1544.pdf
_au_cases_vic_VCAT_2013_644.pdf
_au_cases_vic_VCAT_2020_1073.pdf
_au_cases_vic_VCAT_2012_1570.pdf
_au_cases_vic_VCAT_2018_150.pdf
_au_cases_vic_VCAT_2011_999.pdf
_au_cases_vic_VCAT_2018_408.pdf
_au_cases_vic_VCAT_2007_664.pdf
_au_cases_vic_VCAT_

_au_cases_vic_VCAT_2008_918.pdf
_au_cases_vic_VCAT_2016_1671.pdf
_au_cases_vic_VCAT_2006_1216.pdf
_au_cases_vic_VCAT_2003_956.pdf
_au_cases_vic_VCAT_2020_839.pdf
_au_cases_vic_VCAT_2011_311.pdf
_au_cases_vic_VCAT_2013_2159.pdf
_au_cases_vic_VCAT_2006_1565.pdf
_au_cases_vic_VCAT_2020_1153.pdf
_au_cases_vic_VCAT_2007_340.pdf
_au_cases_vic_VCAT_2006_1972 (1).pdf
_au_cases_vic_VCAT_2008_919.pdf
_au_cases_vic_VCAT_2022_726.pdf
_au_cases_vic_VCAT_2008_266.pdf
_au_cases_vic_VCAT_2014_1409.pdf
_au_cases_vic_VCAT_2013_1861.pdf
_au_cases_vic_VCAT_2018_1862.pdf
_au_cases_vic_VCAT_2012_1097.pdf
_au_cases_vic_VCAT_2007_750.pdf
_au_cases_vic_VCAT_2018_676.pdf
_au_cases_vic_VCAT_2006_1363.pdf
_au_cases_vic_VCAT_2018_1335.pdf
_au_cases_vic_VCAT_2010_2054.pdf
_au_cases_vic_VCAT_2017_544.pdf
_au_cases_vic_VCAT_2008_853.pdf
_au_cases_vic_VCAT_2020_580.pdf
_au_cases_vic_VCAT_2009_2649.pdf
_au_cases_vic_VCAT_2011_1165.pdf
_au_cases_vic_VCAT_2022_1499.pdf
_au_cases_vic_VCAT_2022_724.pdf
_au_cases_vic_VCAT_2

In [181]:
order

['1 The respondent must pay the applicants $6,776.',
 '2 The respondent must reimburse the applicants, the application fee paid by',
 'the applicants of $ 212.50.',
 '3 The respondent must pay the applicants’ costs of $ 1,800.',
 '1. The Applicant must pay the Respondent s $45,772.80 .',
 '2. Liberty to apply on the question of costs and interest, provided such',
 'liberty is exercised within 21 days after receipt  of these orders.',
 '1. The proceeding is reinstated.',
 '2. The respondent must pay the applicants the sum of $ 5,170.00 .',
 '3. Costs are reserved  with liberty to apply .',
 '4. Any submission as to costs by either party must be in writing, filed with the',
 'Tribunal and served on the other party by 20 March 2018.',
 '5. Any determination as to costs will be made in Chambers.',
 'B W Thomas',
 '1 Within seven days the parties must bring in short minutes to give effect to',
 'these reasons.',
 '2 Adjourned to further hearing 9.30 am on 17 December 2014 at 55 King',
 'Str